In [49]:
import pandas as pd
import numpy as np
from missingpy import MissForest
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler

In [2]:
#df = pd.read_csv("Dataset/heart_target.csv")
df = pd.read_csv("Dataset/housePrice_SalePrice.csv")

In [71]:
def preprocess4ensemble(df):
    print("preprocess4ensemble")
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    X_num = X.select_dtypes(include="number")
    X_cat = X.select_dtypes(include="object")

    #Check if all right
    if not(X_num.shape[1]+X_cat.shape[1] == X.shape[1]) or not(X_num.shape[0] == X_cat.shape[0] and  X_cat.shape[0] == X.shape[0]):
        print("categorical and numerical seperation operation has a problem")

    for cat_col in X_cat.columns:
        X_cat = pd.concat([X_cat,pd.get_dummies(X_cat[cat_col],dummy_na=False)],axis=1)
        del X_cat[cat_col]

    imputer = MissForest()
    X_imputed = pd.DataFrame(imputer.fit_transform(X_num))
    X_imputed.columns = X_num.columns
    del X_num

    X = pd.concat([X_imputed,X_cat],axis=1)
    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(X, y)
    return X_res, y_res

def preprocess4xgb(df):
    print("preprocess4xgb")
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    X_num = X.select_dtypes(include="number")
    X_cat = X.select_dtypes(include="object")

    #Check if all right
    if not(X_num.shape[1]+X_cat.shape[1] == X.shape[1]) or not(X_num.shape[0] == X_cat.shape[0] and  X_cat.shape[0] == X.shape[0]):
        print("categorical and numerical seperation operation has a problem")

    for cat_col in X_cat.columns:
        X_cat = pd.concat([X_cat,pd.get_dummies(X_cat[cat_col],dummy_na=False)],axis=1)
        del X_cat[cat_col]

#     imputer = MissForest()
#     X_imputed = pd.DataFrame(imputer.fit_transform(X_num))
#     X_imputed.columns = X_num.columns
#     del X_num
    def oversampling(X_train,y_train):
        rus = RandomOverSampler(return_indices=True)
        X_resampled, y_resampled, idx_resampled = rus.fit_resample(X_train, y_train)
        X_resampled = pd.DataFrame(X_resampled)
        y_resampled = pd.Series(y_resampled)
        X_resampled.columns = X_train.columns
        return X_resampled,y_resampled
    
    X = pd.concat([X_imputed,X_cat],axis=1)
    print(type(X))
    ros = RandomOverSampler(random_state=42)
    X, y = X.fillna(10000000000), y
    X, y = oversampling(X, y)
    X, y = X.replace(10000000000, np.nan), y.replace(10000000000, np.nan)
    
    return X, y

def preprocess4normal(df):
    print("preprocess4normal")
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    X_num = X.select_dtypes(include="number")
    X_cat = X.select_dtypes(include="object")

    #Check if all right
    if not(X_num.shape[1]+X_cat.shape[1] == X.shape[1]) or not(X_num.shape[0] == X_cat.shape[0] and  X_cat.shape[0] == X.shape[0]):
        print("categorical and numerical seperation operation has a problem")

    for cat_col in X_cat.columns:
        X_cat = pd.concat([X_cat,pd.get_dummies(X_cat[cat_col],dummy_na=False)],axis=1)
        del X_cat[cat_col]

    #Impute nan
    imputer = MissForest()
    X_imputed = pd.DataFrame(imputer.fit_transform(X_num))
    X_imputed.columns = X_num.columns
    del X_num
    
    #Scaling
    scaled_features = StandardScaler().fit_transform(X_imputed.values)
    scaled_features_df = pd.DataFrame(scaled_features, index=X_imputed.index, columns=X_imputed.columns)

    X = pd.concat([scaled_features_df,X_cat],axis=1)
    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(X, y)
    return X_res, y_res

def preprocess(df,algo_type):
    if "RandomForest" in str(algo_type).split("(")[0] or "GradientBoosting" in str(algo_type).split("(")[0]:
        return preprocess4ensemble(df)
    elif "XGB" in str(algo_type).split("(")[0]:
        return preprocess4xgb(df)
    else:
        return preprocess4normal(df)

In [72]:
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.svm import LinearSVR, LinearSVC, SVC, SVR
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from xgboost import XGBRegressor, XGBClassifier

In [75]:
test = XGBRegressor()
result = preprocess(df,test)
print(result[0].shape,result[1].shape)

preprocess4xgb
<class 'pandas.core.frame.DataFrame'>
(13260, 289) (13260,)


In [58]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_num = X.select_dtypes(include="number")
X_cat = X.select_dtypes(include="object")

#Check if all right
if not(X_num.shape[1]+X_cat.shape[1] == X.shape[1]) or not(X_num.shape[0] == X_cat.shape[0] and  X_cat.shape[0] == X.shape[0]):
    print("categorical and numerical seperation operation has a problem")

for cat_col in X_cat.columns:
    X_cat = pd.concat([X_cat,pd.get_dummies(X_cat[cat_col],dummy_na=False)],axis=1)
    del X_cat[cat_col]

#Impute nan
imputer = MissForest()
X_imputed = pd.DataFrame(imputer.fit_transform(X_num))
X_imputed.columns = X_num.columns
del X_num

#Scaling
scaled_features = StandardScaler().fit_transform(X_imputed.values)
scaled_features_df = pd.DataFrame(scaled_features, index=X_imputed.index, columns=X_imputed.columns)

#Remove Extreme
scaled_features_df

Iteration: 0
Iteration: 1
Iteration: 2


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,-1.730865,0.073375,-0.252825,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.510061,0.575425,...,0.351000,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777
1,-1.728492,-0.872563,0.390449,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.574854,1.171992,...,-0.060731,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439
2,-1.726120,0.073375,-0.124170,0.073480,0.651479,-0.517200,0.984752,0.830215,0.321862,0.092907,...,0.631726,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777
3,-1.723747,0.309859,-0.467250,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.574854,-0.499274,...,0.790804,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655
4,-1.721374,0.073375,0.561989,0.375148,1.374795,-0.517200,0.951632,0.733308,1.362495,0.463568,...,1.698485,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,2.100892,0.138777
5,-1.719002,-0.163109,0.604874,0.360616,-0.795151,-0.517200,0.719786,0.491040,-0.574854,0.632450,...,0.032844,-0.432931,-0.251539,-0.359325,10.802446,-0.270208,-0.068692,1.323736,1.360892,0.891994
6,-1.716629,-0.872563,0.176024,-0.043379,1.374795,-0.517200,1.084115,0.975575,0.454708,2.029558,...,0.762732,1.283007,0.156111,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.620891,-0.614439
7,-1.714256,0.073375,0.683782,-0.013513,0.651479,0.381743,0.057371,-0.574938,0.753614,0.910994,...,0.051559,1.123385,2.375537,3.372372,-0.116339,-0.270208,-0.068692,0.618024,1.730892,0.891994
8,-1.711883,-0.163109,-0.853215,-0.440659,0.651479,-0.517200,-1.333700,-1.689368,-0.574854,-0.973018,...,-0.023301,-0.033876,-0.704483,2.995929,-0.116339,-0.270208,-0.068692,-0.087688,-0.859110,0.138777
9,-1.709511,3.147673,-0.896099,-0.310370,-0.795151,0.381743,-1.068734,-1.689368,-0.574854,0.893448,...,-1.253816,-0.752176,-0.644091,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.969111,0.138777


In [43]:
X,y = preprocess4xgb(df)

<class 'pandas.core.frame.DataFrame'>


In [146]:
imp = SimpleImputer(missing_values = np.nan, strategy='mean')
for col in X_num.columns:
    print(X_num[col].isna().value_counts())
X_num.info()

KeyError: 'True'

In [ ]:
#oversampling(<10%)
def Oversampling():
  from sklearn.datasets import make_classification
  from sklearn.decomposition import PCA
  from imblearn.over_sampling import SMOTE
  # 모델설정
  global x_train, y_train
  sm = SMOTE(ratio='auto', kind='regular')
  x_train, y_train = sm.fit_sample(x_train,list(y_train))

onehot_col = []
#onehotencoding
def OnehotEncoding():
  global dataset
  global exclude_col 
  dataset = pd.get_dummies(dataset, columns=onehot_col)

def scaling():
  from sklearn.preprocessing import StandardScaler
  global x_train
  scaler = StandardScaler()
  x_train = scaler.fit_transform(x_train)

'''
#missing value(not working with string, so we first use onehotencoding)
def fill():
  MICE = MiceImputer()
  global dataset
  dataset = MICE.fit(dataset).transform(dataset)
'''
OnehotEncoding()
fill()
x = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
Oversampling()
scaling()
